In [0]:
%tensorflow_version 2.x
import tensorflow
import tensorflow.keras as keras
import tensorflow.keras.utils
from tensorflow.keras.layers import LSTM, Input, Dense, TimeDistributed, Masking, Bidirectional
from tensorflow.keras.models import Model

TensorFlow 2.x selected.


In [0]:
tensorflow.__version__

'2.1.0'

# General

In [0]:
alphabet = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'.,-()/") # %:;&#
alphabet.insert(0, chr(0)) # '\x00' character, i.e., ord(0) to label concatenate

# Model

In [0]:
timesteps = None
input_features = 3
lstm_output = 512
alphabet_output_size = len(alphabet)
eoc_output_size = 1
bow_output_isze = 1

In [0]:
lstm_input = Input(shape=(timesteps, input_features), name='stroke_input')
masking_layer =  Masking(mask_value=0., input_shape=(timesteps, input_features))(lstm_input)
lstm_layer = Bidirectional(LSTM(lstm_output, return_sequences=True))(masking_layer)

alphabet_output = TimeDistributed(Dense(alphabet_output_size, activation='softmax'), name='alphabet_output')(lstm_layer)
eoc_output = TimeDistributed(Dense(eoc_output_size, activation='sigmoid'), name='eoc_output')(lstm_layer)
bow_output = TimeDistributed(Dense(bow_output_isze, activation='sigmoid'), name='bow_output')(lstm_layer)

In [0]:
losses ={
    'alphabet_output': 'sparse_categorical_crossentropy',
    'eoc_output' : 'mean_squared_error',
    'bow_output' : 'mean_squared_error'
}

In [0]:
model = Model(inputs=[lstm_input], outputs=[alphabet_output, eoc_output, bow_output])
model.compile(loss=losses, optimizer='adam', metrics=['accuracy'])

# Preprocessing

In [0]:
!ls

sample_data


In [0]:
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

TypeError: ignored

In [0]:
training_dataset = np.load('/content/drive/My Drive/deepwriting/deepwriting_training.npz', allow_pickle=True)
validation_dataset = np.load('/content/drive/My Drive/deepwriting/deepwriting_validation.npz', allow_pickle=True)

In [0]:
list(training_dataset.keys())

In [0]:
def pad_to_length(sequences, max_len, two_dimensional=True):
    padded_sequence = []
    for seq in sequences:
        pad_len = max_len - len(seq)
        if two_dimensional:
            padded_seq = np.pad(seq, [(0, pad_len), (0, 0)], mode='constant', constant_values=0)
        else:
            padded_seq = np.pad(seq, (0, pad_len), mode='constant', constant_values=0)
            padded_seq = np.expand_dims(padded_seq, axis=1)
        padded_sequence.append(padded_seq)
    # check whether all lists have actually the same length
    assert len(list(filter(lambda x: x != max_len, [len(seq) for seq in padded_sequence]))) == 0
    return np.array(padded_sequence)

In [0]:
def check_lengths(seq, max_len):
    other_max_length = len(max(seq, key=len))
    assert other_max_length == max_len

## Pad Training input

In [0]:
max_len_training = len(max(training_dataset['strokes'], key=len))
max_len_validation = len(max(validation_dataset['strokes'], key=len))

In [0]:
training_input_padded = pad_to_length(training_dataset['strokes'], max_len_training)
validation_input_padded = pad_to_length(validation_dataset['strokes'], max_len_validation)

## Pad char labels

In [0]:
check_lengths(training_dataset['char_labels'], max_len_training)
check_lengths(validation_dataset['char_labels'], max_len_validation)

In [0]:
training_char_labels_padded = pad_to_length(training_dataset['char_labels'], max_len_training, False)
validation_char_labels_padded = pad_to_length(validation_dataset['char_labels'], max_len_validation, False)

In [0]:
classes = len(training_dataset['alphabet'])
#char_labels_padded = tensorflow.keras.utils.to_categorical(char_labels_padded, num_classes=classes)

## Pad EOC labels

In [0]:
check_lengths(training_dataset['eoc_labels'], max_len_training)
check_lengths(validation_dataset['eoc_labels'], max_len_validation)

In [0]:
training_eoc_labels_padded = pad_to_length(training_dataset['eoc_labels'], max_len_training, False)
validation_eoc_labels_padded = pad_to_length(validation_dataset['eoc_labels'], max_len_validation, False)

## Pad BOW labels

In [0]:
check_lengths(training_dataset['soc_labels'], max_len_training)
check_lengths(validation_dataset['soc_labels'], max_len_validation)

In [0]:
training_soc_labels_padded = pad_to_length(training_dataset['soc_labels'], max_len_training, False)
validation_soc_labels_padded = pad_to_length(validation_dataset['soc_labels'], max_len_validation, False)

# Training

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [0]:
monitor = 'val_alphabet_output_accuracy'
mode='max'

modelCheckpoint = ModelCheckpoint('./best_model.hdf5', monitor=monitor, save_best_only='True', mode=mode)
earlyStopping = EarlyStopping(monitor=monitor, patience=10, mode=mode)
reduceLROnPlateau = ReduceLROnPlateau(monitor=monitor, factor=0.1, patience=4, mode=mode)

In [0]:
model.fit(
    {
        'stroke_input': training_input_padded
    },
    {
        'alphabet_output': training_char_labels_padded,
        'eoc_output': training_eoc_labels_padded,
        'bow_output': training_soc_labels_padded
    },
    callbacks=[modelCheckpoint, earlyStopping, reduceLROnPlateau],
    epochs=100,
    validation_data=(
        {
            'stroke_input': validation_input_padded
        },
        {
            'alphabet_output': validation_char_labels_padded,
            'eoc_output': validation_eoc_labels_padded,
            'bow_output': validation_soc_labels_padded
        }
    )
)

In [0]:
training_dataset['strokes']